# HKL Interface Cycling

This notebook takes two materials, cycles through Miller indices and constructs a coherent interface using ZSL algorithm, optimizes it with a relaxer and calculator, and then calculates the delta energy for the interface. The delta energy is then plotted as a function of the Miller index.

In [3]:
SETTINGS = {
    "MAX_H": 2,
    "MAX_K": 2,
    "MAX_L": 2,
}

SUBSTRATE_PARAMETERS = {
    "MATERIAL_INDEX": 0,  # the index of the material in the materials_in list
    # TODO: change for looping
    "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 6,  # in layers
}

LAYER_PARAMETERS = {
    "MATERIAL_INDEX": 1, # the index of the material in the materials_in list
    # TODO: change for looping
    "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 1,  # in layers
}

INTERFACE_PARAMETERS = {
    "DISTANCE_Z": 3.0, # in Angstroms
    "MAX_AREA": 50, # in Angstroms^2
}

ZSL_PARAMETERS = {
    "MAX_AREA": INTERFACE_PARAMETERS["MAX_AREA"],  # The area to consider in Angstrom^2
    "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
    "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
    "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
    "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
}
RELAXATION_PARAMETERS = {
    "FMAX": 0.018,
}


In [4]:
from src.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from src.utils import to_pymatgen

def create_interfaces(materials, settings):
    print("Creating interfaces...")
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=materials[settings["SUBSTRATE_PARAMETERS"]["MATERIAL_INDEX"]],
        film_structure=materials[settings["LAYER_PARAMETERS"]["MATERIAL_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Find terminations
    cib._find_terminations()
    terminations = cib.terminations

    # Create interfaces for each termination
    interfaces = {}
    for termination in terminations:
        interfaces[termination] = []
        for interface in cib.get_interfaces(
            termination,
            gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
            film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
            substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
            in_layers=True,
        ):
            # Wrap atoms to unit cell
            interface["interface"].make_supercell((1,1,1), to_unit_cell=True)
            interfaces[termination].append(interface)
    return interfaces, terminations


# Sort interfaces by the specified strain mode and number of sites
def sort_interfaces(interfaces, terminations, strain_mode="mean_abs_strain"):
    sorted_interfaces = {}
    for termination in terminations:
        sorted_interfaces[termination] = sorted(
            interfaces[termination], key=lambda x: (x[strain_mode], x["interface"].num_sites)
        )
    return sorted_interfaces



from src.utils import ase_to_poscar, pymatgen_to_ase


def relax_structure(atoms, optimizer, calculator):
    # Set up the interface for relaxation
    ase_structure = atoms.copy()
    ase_structure.set_calculator(calculator)
    dyn = optimizer(ase_structure)
    dyn.run(fmax=RELAXATION_PARAMETERS["FMAX"])
    
    # Extract results
    ase_original_interface = pymatgen_to_ase(interface)
    ase_original_interface.set_calculator(calculator)
    ase_final_interface = ase_structure
    relaxed_energy = ase_structure.get_total_energy()
    
    # Print out the final relaxed structure and energy
    print('Original structure:\n', ase_to_poscar(ase_original_interface))
    print('\nRelaxed structure:\n', ase_to_poscar(ase_final_interface))
    print(f"The final energy is {float(relaxed_energy):.3f} eV.")
    
    return ase_final_interface, relaxed_energy


def filter_atoms_by_tag(atoms, material_index):
    """Filter atoms by their tag, corresponding to the material index."""
    return atoms[atoms.get_tags() == material_index]

def get_effective_delta_energy(interface, substrate, layer, calculator):
    """Get the effective energy of the atoms."""
    substrate.set_calculator(calculator)
    layer.set_calculator(calculator)
    
    effective_substrate_energy = substrate.get_total_energy()/substrate.get_global_number_of_atoms()
    effective_layer_energy = layer.get_total_energy()/layer.get_global_number_of_atoms()
    effective_relaxed_energy = interface.get_total_energy()/interface.get_global_number_of_atoms()
    print(f"Bulk substrate energy per atom: {effective_substrate_energy:.3f} eV")
    print(f"Free layer energy per atom: {effective_layer_energy:.3f} eV")
    print(f"Interface energy per atom: {effective_relaxed_energy:.3f} eV")
    
    # Calculate the effective interface energy
    
    # number of atoms composing the interface: total, substrate's, layer's
    number_of_interface_atoms = interface.get_global_number_of_atoms()
    number_of_substrate_atoms = filter_atoms_by_tag(interface, SUBSTRATE_PARAMETERS["MATERIAL_INDEX"]).get_global_number_of_atoms()
    number_of_layer_atoms = filter_atoms_by_tag(interface, SUBSTRATE_PARAMETERS["MATERIAL_INDEX"]).get_global_number_of_atoms()
    area = interface.get_volume() / interface.cell[2, 2]
    
    # The formula is given by: (E_interface - E_substrate_bulk * N_substrate - E_layer_free * N_layer) / (2 * A * N_interface), where A is the area of the interface, and N_interface = N_substrate + N_layer
    # Derived from: https://www.sciencedirect.com/science/article/pii/S2589152922000485
    effective_interface_delta_energy = (relaxed_energy * number_of_interface_atoms - effective_substrate_energy * number_of_substrate_atoms - effective_layer_energy * number_of_layer_atoms) / (2 * area * number_of_interface_atoms)
    
    print(f"Effective interface delta energy: {effective_interface_delta_energy:.3f} eV/Å^2 ({effective_interface_delta_energy / 0.16:.3f} J/m^2)")
    return effective_interface_delta_energy

In [7]:
import os
import json
from ase.optimize import BFGS
from ase.calculators.emt import EMT



materials_in = []
input_folder = "materials_list"
for file in os.listdir(input_folder):
    with open(f"{input_folder}/{file}", "r") as f:
        data = f.read()
        materials_in.append(json.loads(data))
        
if "materials_in" in globals():
    pymatgen_materials = [to_pymatgen(item) for item in materials_in]
for material in pymatgen_materials:
    print(material, "\n")
    
    
interfaces, terminations = create_interfaces(
    materials=pymatgen_materials,
    settings={
        "SUBSTRATE_PARAMETERS": SUBSTRATE_PARAMETERS,
        "LAYER_PARAMETERS": LAYER_PARAMETERS,
        "ZSL_PARAMETERS": ZSL_PARAMETERS,
        "INTERFACE_PARAMETERS": INTERFACE_PARAMETERS,
    }
)

sorted_interfaces = sort_interfaces(interfaces, terminations)

termination_index = 0
interface_index = 0

termination = terminations[termination_index]

interface = sorted_interfaces[termination][interface_index]["interface"]


calculator = EMT()
ase_interface = pymatgen_to_ase(interface)
ase_final_interface, relaxed_energy = relax_structure(ase_interface, optimizer=BFGS, calculator=calculator)

ase_substrate = pymatgen_to_ase(pymatgen_materials[SUBSTRATE_PARAMETERS["MATERIAL_INDEX"]])
ase_layer = pymatgen_to_ase(pymatgen_materials[LAYER_PARAMETERS["MATERIAL_INDEX"]])


effective_delta_energy = get_effective_delta_energy(ase_final_interface, ase_substrate, ase_layer, calculator)

Full Formula (Ni1)
Reduced Formula: Ni
abc   :   2.460000   2.460000   2.460000
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (1)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Ni      0    0    0 

Full Formula (C2)
Reduced Formula: C
abc   :   2.467291   2.467291  20.000000
angles:  90.000000  90.000000 120.000000
pbc   :       True       True       True
Sites (2)
  #  SP           a         b    c
---  ----  --------  --------  ---
  0  C     0         0           0
  1  C     0.333333  0.666667    0 

Creating interfaces...
      Step     Time          Energy         fmax
BFGS:    0 19:16:29        1.384447        0.2831
BFGS:    1 19:16:29        1.382948        0.2530
BFGS:    2 19:16:29        1.374722        0.2612
BFGS:    3 19:16:29        1.372230        0.3067
BFGS:    4 19:16:29        1.364045        0.3518
BFGS:    5 19:16:29        1.351208        0.3386
BFGS:    6 19:16:29        1.306172        0.6166
BFGS:    7 19:16: